Accessing GFS data from Amazon S3 bucket and analyzing variables of note (precip, soil moisture)

In [ ]:
import fsspec
import xarray as xr
import s3fs


In [ ]:
#attmepting remotely open the file to "stream" it, or only access certain parts. 
s3_gfs_nc_path = "s3://noaa-gfs-bdp-pds/gfs.20250623/00/atmos/gfs.t00z.sfcanl.nc"
fs = fsspec.filesystem("s3", anon=True)

with fs.open(s3_gfs_nc_path) as f:
    ds = xr.open_dataset(s3_gfs_nc_path, engine="h5netcdf") #Changed to h5netcdf backend because the netcdf4 backend does not support remotely opening files. 
    

In [ ]:
hour = "00"
s3_gfs_nc_path = f"s3://noaa-gfs-bdp-pds/gfs.20250623/{hour}/atmos/gfs.t{hour}z.sfcanl.nc"
fs = fsspec.filesystem("s3", anon=True)

fsspec_caching = {
    "cache_type": "blockcache",  # block cache stores blocks of fixed size and uses eviction using a LRU strategy.
    "block_size": 8
    * 1024
    * 1024,  # size in bytes per block, adjust depends on the file size but the recommended size is in the MB
}

ds = xr.open_dataset(fs.open(s3_gfs_nc_path, **fsspec_caching), engine="h5netcdf")
ds

In [ ]:
ds.attrs[""]

In [ ]:
ds_us = ds.sel(grid_xt=slice(((180-125.5)+180), ((180-64)+180)),grid_yt=slice(50.2, 22))

In [ ]:
ds_us['tprcp']

In [ ]:
ds_us_tpcrp_00 = ds_us['tprcp'].where(ds_us['tprcp'] != ds_us['tprcp'].attrs['missing'])

In [ ]:
ds_us_tpcrp_00.plot(robust=True)

Now I will try to pull all the 6 hourly files and add them together to get the full 6-23 precipitation, given prism is the full day in mm. 

In [ ]:
hour = "06"
s3_gfs_nc_path = f"s3://noaa-gfs-bdp-pds/gfs.20250623/{hour}/atmos/gfs.t{hour}z.sfcanl.nc"
fs = fsspec.filesystem("s3", anon=True)

fsspec_caching = {
    "cache_type": "blockcache",  # block cache stores blocks of fixed size and uses eviction using a LRU strategy.
    "block_size": 8
    * 1024
    * 1024,  # size in bytes per block, adjust depends on the file size but the recommended size is in the MB
}

ds_06 = xr.open_dataset(fs.open(s3_gfs_nc_path, **fsspec_caching), engine="h5netcdf")
ds_06

In [ ]:
ds_us_06 = ds_06.sel(grid_xt=slice(((180-125.5)+180), ((180-64)+180)),grid_yt=slice(50.2, 22))
ds_us_tpcrp_06 = ds_us_06['tprcp'].where(ds_us_06['tprcp'] != ds_us_06['tprcp'].attrs['missing'])
ds_us_tpcrp_06.plot(robust=True)

In [ ]:
hour = "12"
s3_gfs_nc_path = f"s3://noaa-gfs-bdp-pds/gfs.20250623/{hour}/atmos/gfs.t{hour}z.sfcanl.nc"
fs = fsspec.filesystem("s3", anon=True)

fsspec_caching = {
    "cache_type": "blockcache",  # block cache stores blocks of fixed size and uses eviction using a LRU strategy.
    "block_size": 8
    * 1024
    * 1024,  # size in bytes per block, adjust depends on the file size but the recommended size is in the MB
}

ds_12 = xr.open_dataset(fs.open(s3_gfs_nc_path, **fsspec_caching), engine="h5netcdf")
ds_12

In [ ]:
ds_us_12 = ds_12.sel(grid_xt=slice(((180-125.5)+180), ((180-64)+180)),grid_yt=slice(50.2, 22))
ds_us_tpcrp_12 = ds_us_12['tprcp'].where(ds_us_12['tprcp'] != ds_us_12['tprcp'].attrs['missing'])
ds_us_tpcrp_12.plot(robust=True)

In [ ]:
hour = "18"
s3_gfs_nc_path = f"s3://noaa-gfs-bdp-pds/gfs.20250623/{hour}/atmos/gfs.t{hour}z.sfcanl.nc"
fs = fsspec.filesystem("s3", anon=True)

fsspec_caching = {
    "cache_type": "blockcache",  # block cache stores blocks of fixed size and uses eviction using a LRU strategy.
    "block_size": 8
    * 1024
    * 1024,  # size in bytes per block, adjust depends on the file size but the recommended size is in the MB
}

ds_18 = xr.open_dataset(fs.open(s3_gfs_nc_path, **fsspec_caching), engine="h5netcdf")
ds_18

In [ ]:
ds_us_18 = ds_18.sel(grid_xt=slice(((180-125.5)+180), ((180-64)+180)),grid_yt=slice(50.2, 22))
ds_us_tpcrp_18 = ds_us_18['tprcp'].where(ds_us_18['tprcp'] != 0)
ds_us_tpcrp_18.plot(robust=True)

In [ ]:
ds_tpcrp_total = xr.concat([ds_us_tpcrp_00, ds_us_tpcrp_06, ds_us_tpcrp_12, ds_us_tpcrp_18], dim="time")

In [ ]:
ds_tpcrp_total

In [ ]:
ds_tpcrp_total = ds_tpcrp_total.sum(dim="time")

In [ ]:
ds_tpcrp_total

In [ ]:
ds_tpcrp_total.plot(col="time", robust=True)

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

projection = ccrs.PlateCarree()

fig, axis = plt.subplots(1, 1, subplot_kw=dict(projection=ccrs.Orthographic(-90, 30)))


ds_tpcrp_total['total_tpcrp'].plot(ax=axis, transform=ccrs.PlateCarree(), cbar_kwargs={"orientation": "horizontal", "shrink": 0.7}, robust=True)

axis.coastlines()

In [ ]:
ds_tpcrp_total

In [ ]:
ds_tprcp_total_4326 = ds_tpcrp_total.rio.write_crs("EPSG:4326")
ds_tprcp_total_4326_dims = ds_tprcp_total_4326.rio.set_spatial_dims("grid_xt", "grid_yt")
ds_tprcp_5070 = ds_tprcp_total_4326_dims.rio.reproject("EPSG:5070")

In [ ]:
ds_tprcp_5070.plot(robust=True)

GDAS tprcp is in units of kg/m**2, which should be the same as mm. 

In [ ]:
ds_tprcp_5070_units = ds_tprcp_5070 * 10000

In [ ]:
ds_tprcp_5070_units.plot(robust=True)

In [ ]:
ds_tprcp_5070_units.to_netcdf("data/gfs/gfs_06232025_tprcp_5070.nc")